In [1]:
import pandas as pd
import json

In [2]:
source_dir = 'source'
result_dir = 'result'

In [3]:
# Use ExcelFile to get the sheet names
file_name = f"{source_dir}/요금제 검색 entity.xlsx"
xls = pd.ExcelFile(file_name)

# Get the list of sheet names
sheet_names = xls.sheet_names

find_sheet = '요금제'
sheet = [s for s in sheet_names if find_sheet in s]

df = pd.read_excel(file_name, sheet_name=sheet[0], header=1)
df = df.where(pd.notnull(df), None)

for col in df.columns:
    if '라인업유사' in col:
        df[col] = df[col].astype('str')
    if 'VAT' in col:
        df[col] = df[col].fillna(0).astype('int').astype('str')
df.head(5)

,상품명,상품ID,template 카테고리,"요금제 라인업, 혜택 라인업",월정액(VAT 제외),세대,요금제유사어,요금제유사어2,요금제유사어3,라인업1,...,라인업유사3,라인업유사4,라인업유사5,라인업유사6,라인업유사7,월정액(VAT포함),요금제_kw,요금제_kw2,작업,Unnamed: 26
0,0 청년 다이렉트 62(넷플릭스),NA00008731,휴대폰 요금제,0청년 다이렉트 넷플릭스 요금제,56364,None,None,None,OTT,티다,...,1934,20대,30대,청년,넷플,62000,62.0,무제한,티다청년62,None
1,0 청년 다이렉트 69(넷플릭스),NA00008730,휴대폰 요금제,0청년 다이렉트 넷플릭스 요금제,62728,None,None,None,OTT,티다,...,1934,20대,30대,청년,넷플,69000,69.0,None,티다청년69,None
2,0 청년 89(넷플릭스),NA00008728,휴대폰 요금제,0청년 넷플릭스 요금제,80909,None,None,None,OTT,None,...,1934,20대,30대,청년,넷플,88999,89.0,None,None,None
3,0 청년 99(넷플릭스),NA00008727,휴대폰 요금제,0청년 넷플릭스 요금제,90000,None,None,None,OTT,None,...,1934,20대,30대,청년,넷플,99000,99.0,None,None,None
4,0 청년 109(넷플릭스),NA00008726,휴대폰 요금제,0청년 넷플릭스 요금제,99091,None,None,None,OTT,None,...,1934,20대,30대,청년,넷플,109000,109.0,None,None,None


In [4]:
from tworld.tworld_funcs import categorize_and_convert, create_lineup_prompt, create_plans_prompt

# Convert the DataFrame to categorized JSON
mobile_plans = categorize_and_convert(df)

# Convert dictionary to JSON and save to a file
with open(f'{result_dir}/mobile_plans.json', 'w', encoding='utf-8') as json_file:
    json.dump(mobile_plans, json_file, ensure_ascii=False, indent=4)

# Optionally, print the JSON structure
mobile_plans


2024-11-01 10:44:34,509 - tworld_funcs> categorize_and_convert> 103 - INFO - lineup_similarity_cols = ['라인업1', '라인업2', '라인업3', '라인업4', '라인업5']
2024-11-01 10:44:34,509 - tworld_funcs> categorize_and_convert> 107 - INFO - plan_similarity_cols = ['요금제유사어', '요금제유사어2', '요금제유사어3']


{'lineup': [{'name': '0청년 다이렉트 넷플릭스 요금제',
   'lineup_similarities': ['언택트', '20대', '티다', 'MZ', '온라인'],
   'products': [{'product_name': '0 청년 다이렉트 62(넷플릭스)',
     'details': {'상품ID': 'NA00008731',
      'template 카테고리': '휴대폰 요금제',
      '월정액(VAT 제외)': '56364',
      '세대': None,
      '유사어': ['OTT'],
      '월정액(VAT 포함)': '62000'}},
    {'product_name': '0 청년 다이렉트 69(넷플릭스)',
     'details': {'상품ID': 'NA00008730',
      'template 카테고리': '휴대폰 요금제',
      '월정액(VAT 제외)': '62728',
      '세대': None,
      '유사어': ['OTT'],
      '월정액(VAT 포함)': '69000'}},
    {'product_name': '0 청년 다이렉트 76(넷플릭스)',
     'details': {'상품ID': 'NA00008729',
      'template 카테고리': '휴대폰 요금제',
      '월정액(VAT 제외)': '69091',
      '세대': None,
      '유사어': None,
      '월정액(VAT 포함)': '0'}}]},
  {'name': '0청년 넷플릭스 요금제',
   'lineup_similarities': ['20대', 'MZ'],
   'products': [{'product_name': '0 청년 89(넷플릭스)',
     'details': {'상품ID': 'NA00008728',
      'template 카테고리': '휴대폰 요금제',
      '월정액(VAT 제외)': '80909',
      '세대': Non

In [9]:
with open(f'{result_dir}/mobile_plans.json', 'rb') as f:
    mobile_plan_data = json.load(f) 

# Create the prompt for mobile plans from json
lineup_prompt = create_lineup_prompt(mobile_plan_data)
# print(lineup_prompt)

plans_prompt = create_plans_prompt(mobile_plan_data)
# print(plans_prompt)

# Save the prompt to a text file
output_file = 'result/mobile_plans_prompt.txt'  # Specify the output file name

plans_lineup_prompt = f'{lineup_prompt}{plans_prompt}'

with open(output_file, 'w', encoding='utf-8') as f:
    f.write(plans_lineup_prompt)

print(f"The prompt has been saved to {output_file}.")

# Print the prompt
print(plans_lineup_prompt)


The prompt has been saved to result/mobile_plans_prompt.txt.
### Below is a list of mobile lineups grouped by their respective similar words or phrases :
1. **similarities:** 언택트, 티다, 온라인
  - **lineups:** LTE 다이렉트플랜, 0청년 다이렉트플랜, 다이렉트 넷플릭스 요금제, 0청년 다이렉트 넷플릭스 요금제, 다이렉트플랜

2. **similarities:** 20대, MZ
  - **lineups:** 0청년 다이렉트플랜, 0청년 넷플릭스 요금제, 0청년, 0청년 다이렉트 넷플릭스 요금제

3. **similarities:** 5GX
  - **lineups:** 5G 일반요금제

4. **similarities:** 팅, 10대, 중등, 미성년, 고등
  - **lineups:** 5G 청소년 요금제, 주말엔 팅, 0틴플랜

5. **similarities:** 실버, 어르신
  - **lineups:** T플랜 시니어, 5G 시니어 요금제

6. **similarities:** 어린이, 초등
  - **lineups:** 5G ZEM 플랜, ZEM플랜

7. **similarities:** 복지
  - **lineups:** 행복누리, 소리누리, 손누리

8. **similarities:** 청각장애
  - **lineups:** 행복누리, 손누리

9. **similarities:** LTE
  - **lineups:** T플랜

10. **similarities:** 군인
  - **lineups:** 0히어로

11. **similarities:** 청년
  - **lineups:** 0플랜

12. **similarities:** 맞춤, DIY
  - **lineups:** 뉴 T끼리 맞춤형

13. **similarities:** 시각장애
  - **lineups:** 소리누리

### B